In [ ]:
import numpy as np 
from numpy.linalg import LinAlgError

from numba import njit

from scipy.interpolate import RegularGridInterpolator, interp1d, griddata
from scipy.linalg import solve

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

from pricing import bs, heston
from implied_vol import compute_implied_vol
from kernels import gaussian_kernel as kernel 
from calibrating_dupire import Dupire_local_vol


from simulation_functions import *


import sys 
import warnings
warnings.filterwarnings("ignore")

The larger $\Delta_t$ you choose, the higher the regularization parameter $\lambda$ to avoid singularities in the matrices: 
- around $\lambda = 1e-7$ for $\Delta_t$ = 0.01
This means that, as $\Delta_t \rightarrow 0$, we don't need anymore regularization. 

For longer dated maturities, we need to increase the bounds for our grid for the strikes, as more and more particules will go higher and lower. For 10 years, it's good to set $K_\text{min} = 0.01$ and $K_\text{max} = 15$.  

In [ ]:
## Whether to use Black Scholes of Heston Formula
black_scholes = False 

## Parameter for the regularization
lambda_par = 1e-7

## Simulations parameters
nb_X = int(1e4)                      # Number of particles

In [ ]:
# Distinction between training and simulation grid 
# for instance, for Heston: Dupire calibrated on training, simulated on sim

Tmax = 10.                            # Maximum maturity
delta_t = 0.01
nb_T_sim = int(Tmax / delta_t)
nb_T_train = 200

Kmax = 15. 
Kmin = 0.01
nb_K_sim = 200
nb_K_train = 100

kgrid_train = np.linspace(Kmin, Kmax, nb_K_train); tgrid_train = np.linspace(delta_t, Tmax, nb_T_train)
K_train, T_train = np.meshgrid(kgrid_train, tgrid_train) 

kgrid_sim = np.linspace(Kmin, Kmax, nb_K_sim); tgrid_sim  = np.linspace(delta_t, Tmax, nb_T_sim)
K_sim, T_sim = np.meshgrid(kgrid_sim, tgrid_sim) 


## Parameter for the cap
cap = 1e-3


## Dictionary of parameter values 
PARAMETERS = {
    'S0': 1.0,             # Initial Spot
    'r': 0.0,              # Fixed interest rates
    'v0': 0.0045,          # Initial volatility 
    'sigma':0.3,           # Constant volatility (for Black Scholes)
    'kappa': 2.19,         # Speed of the mean reversion
    'theta': 0.17023,      # Long term mean
    'zeta': 1.04,          # Volatility of volatility
    'rho': -0.83,          # correlation between 2 random variables
    'opt_type':1,          # 1 for call
    'N': 1_012,
    'z': 24,

## LSV Model parameters

    'X_0': 1.0,
    'Y_0': 0.0144,         # vol_0 @ 12%
    'mu': 0.0144,
    'lambda': 1,
    'eta': 0.5751,
    'rho_sto': -0.9,
    'cap': cap,
    

    ## Simulation parameters
    'nb_X': nb_X,
    'Tmax': Tmax,
    'delta_t': delta_t,
    'nb_T_sim': nb_T_sim,

    ## Grids 

    'tgrid_train':tgrid_train,
    'kgrid_train':kgrid_train,
    'K_train':K_train,
    'T_train':T_train,

    'K_sim':K_sim,
    'T_sim':T_sim
}

In [ ]:

if black_scholes:
    prices_model = bs(PARAMETERS, train=True)

    sigma_dupire = lambda X: np.ones_like(X[1]) * PARAMETERS['sigma'] # basically a constant
else:
    prices_model = heston(PARAMETERS, True)
    
    local_vol = Dupire_local_vol(prices_model, PARAMETERS)
    sigma_dupire = RegularGridInterpolator((tgrid_train, kgrid_train), local_vol, method='linear') # Interpolation grid

In [ ]:
X_, v_, dW_x, dW_v = initialize_processes(PARAMETERS)

### Diffusion Simulation 

In [ ]:
cond = np.zeros([nb_T_sim, nb_X]) 
L = 100
sigma_f = 0.1

t = 1
X_[:, t] = X_[:, t-1] +  np.sqrt(PARAMETERS['Y_0']) * X_[:, t-1] * sigma_dupire((PARAMETERS['delta_t'], X_[:, t-1])) / np.sqrt(PARAMETERS['Y_0']) * dW_x[:, t]

In [ ]:
Par=PARAMETERS
prices=prices_model
t=Par['tgrid_train'] 
k=Par['kgrid_train'] 
r=Par['r']
K=Par['K_train']

dC_dT = np.gradient(prices, t, axis=0)  # Time derivative
dC_dT = np.where(dC_dT < 0, 1e-10, dC_dT) # can't be 0 given Dupire formula


dC_dK = np.gradient(prices, k, axis=1)  # First strike derivative
d2C_dK2 = np.gradient(dC_dK, k, axis=1) # Second strike derivative

local_vol = np.sqrt((dC_dT) / (0.5 * K**2 * d2C_dK2))

In [ ]:
t=2
beta_hat, m_A_lam = compute_beta(X_, v_, t, L, kernel, lambda_par, sigma_f, PARAMETERS)
m_A_lam.shape

In [ ]:


for t in range(2, PARAMETERS['nb_T_sim']):
    # period 
    beta_hat, m_A_lam = compute_beta(X_, v_, t, L, kernel, lambda_par, sigma_f, PARAMETERS)

    ## Update the memory for the graphs later
    cond[t, :] = m_A_lam.ravel()
    
    # Update step 
    Sig = sigma_dupire((tgrid_sim[t], X_[:, t-1]))
    ## Only forst the initial iterations, given that Dupire's formula isn't great for extra short term 
    
    if np.isnan(Sig).any():
        # mean_value = np.nanmean(Sig)
        # Sig = np.where(np.isnan(Sig), mean_value, Sig)  # Replace NaN values with the mean value
        indices = np.arange(len(Sig))

        # Mask for non-NaN values
        valid_mask = ~np.isnan(Sig)

        # Interpolate the NaN values
        interp_func = interp1d(indices[valid_mask], Sig[valid_mask], kind='linear', fill_value="extrapolate")
        Sig = interp_func(indices)
        
    X_[:, t] = X_[:, t-1] +  np.sqrt(v_[:,t-1]) * X_[:, t-1] * Sig / np.sqrt(m_A_lam.ravel()) * dW_x[:, t]

    # Bounds for the interpolation
    X_[:, t][X_[:, t] > Kmax] = Kmax
    X_[:, t][X_[:, t] < Kmin] = Kmin

    print(f"\rSteps completed: {t}/{nb_T_sim}", end='', flush=True)

### Graphical Insights into the Simulation

In [ ]:
## Plot the diffusion 

mean_ = np.mean(X_, axis=0)
std_ = np.std(X_, axis=0)

confidence_level = 0.95
confidence_interval = std_ * 1.96  # Approximate z-score for 95% confidence


plt.figure() 
for i in range(100):
    plt.plot(X_[i,:], color='skyblue', alpha=0.7)
    
plt.title("Diffusion of " + str(100) + " processes")
plt.ylim(Kmin, 5) 

# Plot mean and confidence interval
plt.plot(mean_, color="red", label="Mean")
plt.fill_between(range(nb_T_sim), mean_ - confidence_interval, mean_ + confidence_interval, color='red', alpha=0.2, label="95% Confidence Interval")
plt.legend()
plt.show() 

In [ ]:
iteration = 100

fig, axes = plt.subplots(1, 4, figsize=(12, 5))

# Plot 1: Histogram of the Conditional Expectation
axes[0].hist(cond[iteration, :].ravel(), bins=50, edgecolor='black')
axes[0].set_title(r'$m_A^\lambda$ distribution')
axes[0].set_xlabel('Conditional Expectation')

# Plot 2: f(x) = E[v|x]
axes[1].scatter(X_[:, iteration-1], cond[iteration, :].ravel())
axes[1].set_title(r'$\mathbb{E}[\nu_t|X_t]$')
axes[1].set_xlabel('Spot')
axes[1].set_ylabel(r'$m_A^\lambda$ values')

# Plot 3: Spot distribution
axes[2].hist(X_[:,iteration], bins=100, edgecolor='black')
axes[2].set_title('Spot Distribution')
axes[2].set_xlabel('Spot')

axes[3].hist(dW_x[:, iteration], bins=100, edgecolor='black')
axes[3].set_title('Brownian for X')

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
i = 5000

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot on the first axis
ax1.plot(X_[i, :], label='$S_t$')
ax1.plot(dW_x[i, :], label='$W^X_t$')
ax1.plot(dW_v[i, :], label='$W^Y_t$')
ax1.legend(loc='lower right')
ax1.set_title('Plot of $S_t$, $W^X_t$, and $W^Y_t$')
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')

# Plot on the second axis
ax2.plot(v_[i, :], label=r'$\nu_t$')
ax2.plot(cond[:, i], label=r'$m_A^\lambda$')
ax2.legend(loc='lower right')
ax2.set_title(r'Plot of $\nu_t$ and $m_A^\lambda$')
ax2.set_xlabel('Time')
ax2.set_ylabel('Value')

# Adjust layout
plt.tight_layout()

### Comparing Model vs Predictions for the Implied Volatility

In [ ]:
k=kgrid_sim[4]
payoffs = np.maximum(X_ - k, 0)
np.sum(payoffs, axis=0).shape

In [ ]:
prices_LSV = np.zeros((nb_T_sim, nb_K_sim))

for i, k in enumerate(kgrid_sim):
    payoffs = np.maximum(X_ - k, 0)  # Compute max(X_t - k_i, 0) for each element
    prices_LSV[:, i] = 1/nb_X * np.sum(payoffs, axis=0)  # Sum over all simulations (nb_x)


In [ ]:
## Plotting the prices 

fig = go.Figure(data=[go.Surface(z=prices_LSV, x=T_sim, y=K_sim, colorscale='Viridis')])

# Add labels
fig.update_layout(
    scene=dict(
        xaxis_title='Time',
        yaxis_title='Strike',
        zaxis_title='Price'
    ),
    title="Model Pricing"
)

# Show the plot
fig.show()

In [ ]:
vimp_LSV = compute_implied_vol(prices_LSV, PARAMETERS, train=False)
vimp_model = compute_implied_vol(prices_model, PARAMETERS, train=True)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(T_sim, K_sim, vimp_LSV, cmap='viridis')

# Add labels
ax.set_xlabel('Time')
ax.set_ylabel('Strike')
ax.set_zlabel('Implied Vol')
plt.title("Implied Volatility Surface") 

# Show the plot
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8, 3))

fig.suptitle(f"Implied Volatility for {nb_X:,} particles", fontsize=16)

first_mat = 1
second_mat = 4
third_mat = 10

axs[0].plot(kgrid_sim, vimp_LSV[np.argmin(np.abs(tgrid_sim - first_mat)), :], label='LSV')
axs[0].plot(kgrid_train, vimp_model[np.argmin(np.abs(tgrid_train - first_mat)), :], label='True Model')
axs[0].set_title(f"Implied Volatility - {first_mat}yr")
axs[0].set_xlabel("Strike")
axs[0].set_ylabel("Implied vol.")
axs[0].set_xlim(0.5, 2)
axs[0].set_ylim(0.1, 0.6)
axs[0].legend()

axs[1].plot(kgrid_sim, vimp_LSV[np.argmin(np.abs(tgrid_sim - second_mat)), :], label='LSV')
axs[1].plot(kgrid_train, vimp_model[np.argmin(np.abs(tgrid_train - second_mat)), :], label='True Model')
axs[1].set_title(f"Implied Volatility - {second_mat}yr")
axs[1].set_xlabel("Strike")
axs[1].set_ylabel("Implied vol.")
axs[1].set_xlim(0.5, 2)
axs[1].set_ylim(0.2, 0.6)
axs[1].legend()


axs[2].plot(kgrid_sim, vimp_LSV[np.argmin(np.abs(tgrid_sim - third_mat)), :], label='LSV')
axs[2].plot(kgrid_train, vimp_model[np.argmin(np.abs(tgrid_train - third_mat)), :], label='True Model')
axs[2].set_title(f"Implied Volatility - {third_mat}yr")
axs[2].set_xlabel("Strike")
axs[2].set_ylabel("Implied vol.")
axs[2].set_xlim(0.5, 2)
axs[2].set_ylim(0.2, 0.6)
axs[2].legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Assuming you have the following variables defined:
# kgrid_sim, vimp_LSV, tgrid_sim, first_mat, kgrid_train, vimp_model, tgrid_train

# Create a figure
fig = go.Figure()
first_mat = 3
# Add traces
fig.add_trace(go.Scatter(
    x=kgrid_sim,
    y=prices_LSV[np.argmin(np.abs(tgrid_sim - first_mat)), :],
    mode='lines',
    name='LSV'
))

fig.add_trace(go.Scatter(
    x=kgrid_train,
    y=prices_model[np.argmin(np.abs(tgrid_train - first_mat)), :],
    mode='lines',
    name='True Model'
))

# Update layout
fig.update_layout(
    title=f"Implied Volatility - {first_mat}yr",
    xaxis_title="Strike",
    yaxis_title="Implied vol.",
    xaxis_range=[0.5, 2],
    yaxis_range=[0.1, 0.6],
    legend_title="Legend"
)

# Show the plot
fig.show()
